<b><font color='red' size='6'>ch10. 데이터 베이스 연동</font><b>

# 1절. SQLite 데이터 베이스 연결
- SQLite 데이터 베이스는 별도의 DBMS없이 SQL을 이용하여 DB엑세스 할 수 있도록 만든 간단한 디스크 기반의 DB 제공
- C 라이브러리
- SQLite는 프로토타입 을 만들 때 사용
- 프로젝트 단계 : 분석 → 설계 → 구현 → 테스트 → 고객에게 배포 → 유지보수단계
-              프로토타입(SQLite) 시제품   완제품(Oracle, MySql, Maria, ...)
- 
- [DB browser for SQLite](https://sqlitebrowser.org/dl/)
## 1.1 SQLite boswer 설치 및 sqlite3 패키지 load

In [2]:
import sqlite3
sqlite3.sqlite_version # sqlite3 패키지의 버전 확인

'3.40.1'

In [2]:
import pandas as pd
pd.__version__ # 다른 패키지 버전 확인 하는 방법

'1.5.3'

In [3]:
import numpy as np
np.__version__

'1.23.5'

## 1.2 데이터베이스 생성 및 연결
- SQLite로 DB 연결시, DB파일이 있으면 연결 되고 , DB 파일이 없으면 빈DB 파일 생성됨

In [3]:
# DB 연결
conn = sqlite3.connect('data/ch10_data.db')
conn

In [6]:
# 커서 객체 생성 : 커서는 SQl문 실행시키고, 결과 데이터를 조회하는데 사용하는 객체
cursor = conn.cursor()
cursor

In [7]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

In [8]:
cursor.execute('DROP TABLE MEMBER')

In [9]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

In [8]:
cursor.execute('INSERT INTO MEMBER VALUES (\'홍길동\', 20, \'h@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)
sql = "INSERT INTO MEMBER VALUES ('신길동', 25, 'S@S.COM')"
cursor.execute(sql)
print('수행 결과 행수 :', cursor.rowcount)
cursor.execute('INSERT INTO MEMBER VALUES (\'신림동\', 30, \'s@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)

수행 결과 행수 : 1
수행  결과 행수 : 1


In [9]:
conn.commit() # 反. conn.rollback() DML문에서만 commit

In [10]:
# SQL 전송 결과는 cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER ORDER BY NAME")

In [11]:
# insert, update, delete 실행 결과는 : cursor.rowcount 에 들어가지만
# select 문 실행 결과를 받는 함수 들
    ## fetchone() : 결과를 한행씩 받을 때 (튜플)
    ## getchall() : 결과를 모두 받을 때 (튜플 list)
    ## fetchmany(n) : 결과를 n행 받을 때 (튜플 list)
print(cursor.fetchall())

[('신길동', 25, 'S@S.COM'), ('신림동', 30, 's@h.com'), ('홍길동', 20, 'h@h.com')]


In [12]:
print(cursor.fetchall()) # 한번 소요된 cursor 객체는 다시 fetch 할 수 없음

[]


In [14]:
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members # 튜플 리스트

[('홍길동', 20, 'h@h.com'), ('신길동', 25, 'S@S.COM'), ('신림동', 30, 's@h.com')]

In [16]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('신길동', 25, 'S@S.COM')
('신림동', 30, 's@h.com')


In [17]:
# 한줄씩 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
member_list = []
while True:
    member = cursor.fetchone() # 한줄 SQL문 수행 결과 한 줄 가져오기
    if member is None:
        break
    member_list.append( {'name':member[0],
                        'age':member[1],
                        'email':member[2]})

In [18]:
member_list

[{'name': '홍길동', 'age': 20, 'email': 'h@h.com'},
 {'name': '신길동', 'age': 25, 'email': 'S@S.COM'},
 {'name': '신림동', 'age': 30, 'email': 's@h.com'}]

In [20]:
# 최상위 n행 가져오기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(10):
    print(member)

('홍길동', 20, 'h@h.com')
('신길동', 25, 'S@S.COM')
('신림동', 30, 's@h.com')


In [21]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    def as_dic(self): # insert문 전송시 필요
        return {'name':self.name,
               'age':self.age,
               'email':self.email}
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)
def to_member(*row): #튜플 데이러를 매개변수로 받아 Member형 객체로 return 
    return Member(row[0], row[1], row[2])

In [23]:
dbreadmember = ('홍길동', 20, 'h@h.com')
m = to_member(*dbreadmember) # 튜플 언패킹
print(m)
print(m.as_dic())

홍길동	20	h@h.com
{'name': '홍길동', 'age': 20, 'email': 'h@h.com'}


In [25]:
# DB 검색 결과를 객체 list로
cursor.execute('SELECT * FROM MEMBER')
member_list = [] # SQL문 수행 결과를 담을 객체 list
members = cursor.fetchall() # 튜플 list
# print(members)
for member in members:
    member_list.append(to_member(*member))

In [26]:
type(members[0]), type(member_list[0])

(tuple, __main__.Member)

In [27]:
for member in member_list:
    print(member)

홍길동	20	h@h.com
신길동	25	S@S.COM
신림동	30	s@h.com


In [28]:
cursor.close() # cursor.close()는 생략 가능
conn.close()

## 1.3 SQL구문에 파라미터 사용하기
- qmark(DB에 따라 불가한 경우가 있음)
- named(추천)

In [29]:
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동', '신길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('신길동', 25, 'S@S.COM')]

In [33]:
# SQL문에 파라미터 사용하기 : qmark 방법 이용
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
names = [name1, name2]
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", names)
cursor.fetchall()

검색할 이름1 :신림동
검색할 이름2 :김길동


[('신림동', 30, 's@h.com')]

In [42]:
# 파라미터 사용하기 : named 방법 이용
name = input("검색할 이름은 > ")
cursor.execute("SELECT * FROM MEMBER WHERE NAME=:name", {'name':name})
members= cursor.fetchall()
if len(members):
    print(members)
else:
    print(f"검색 하신 이름 '{name}' 님의 데이터가 없습니다")

검색할 이름은 > 김
검색 하신 이름 김 님의 데이터가 없습니다


In [44]:
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
names = {'name1':name1, 'name2':name2}
# cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)", 
#                {'name1':name1, 'name2':name2})
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)", 
               names)
cursor.fetchall()

검색할 이름1 :김
검색할 이름2 :홍길동


[('홍길동', 20, 'h@h.com')]

In [1]:
# MEMBER 테이블에 입력(사용자로부터 이름, 나이, 메일 을 입력받아 insert)
import sqlite3
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()

In [2]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    def as_dic(self): # insert문 전송시 필요
        return {'name':self.name,
               'age':self.age,
               'email':self.email}
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)
def to_member(*row): #튜플 데이러를 매개변수로 받아 Member형 객체로 return 
    return Member(row[0], row[1], row[2])

In [6]:
try:
    name = input('이름을 입력해주세요 >')
    age = int(input('나이를 입력해주세요 >'))
except ValueError:
    print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
    age = 18
finally:
    email = input('이메일을 입력해주세요 >')
inputdata = {'name':name, 'age':age, 'email':email} # named 방식
inputdata2 = (name, age, email) # qmark 방식
# cursor.execute('INSERT INTO MEMBER VALUES (?, ?, ?)',inputdata2)
cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', inputdata)
conn.commit()
if cursor.rowcount ==1:
    print('저장완료')
# print('저장완료' if cursor.rowcount=1 else '저장실패')

이름을 입력해주세요 >김세동
나이를 입력해주세요 >33
이메일을 입력해주세요 >k@k.com
저장완료


In [8]:
try:
    name = input('이름을 입력해주세요 >')
    age = int(input('나이를 입력해주세요 >'))
except ValueError:
    print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
    age = 18
finally:
    email = input('이메일을 입력해주세요 >')
    
newMember = Member(name, age, email)
# print(newMember.as_dic()) 
cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', newMember.as_dic())
conn.commit()
if cursor.rowcount==1:
    print('입력 성공')

이름을 입력해주세요 >김민준
나이를 입력해주세요 >31
이메일을 입력해주세요 >fn45@naver.com
{'name': '김민준', 'age': 31, 'email': 'fn45@naver.com'}
입력 성공


In [3]:
# member 테이블에 데이터를 n번 입력하고 
# 이름에 0을 입력할 때까지 이름, 나이, 메일을 받아 insert
# 이름에 0을 입력하면 이때 까지 입력한 데이터들을 출력 하고 몇명을 입력했는지 출력
member_list = []
while True:
    try:
        name = input('이름을 입력해주세요 : ')
        if name == '0':
            break
        age = int(input('나이를 입력해주세요 :'))        
        email = input('메일을 입력해주세요 :')
    except ValueError:
        print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
        age = 18
    newMember = {'name':name, 'age':age, 'email':email}
    member_list.append(newMember)
    cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', newMember)
conn.commit()
print('입력 하신 데이터는 ', len(member_list), '명 입니다', sep='')
for member in member_list:
    print("{}\t{}\t{}".format(member['name'],member['age'],member['email']))

이름을 입력해주세요 : 0
입력 하신 데이터는 0명 입니다


In [6]:
member_list = [] # 입력한 member들이 저장될 list 변수 (객체 리스트)
while True:
    try:
        name = input('이름을 입력해주세요 : ')
        if name == '0':
            break
        age = int(input('나이를 입력해주세요 :'))        
        email = input('메일을 입력해주세요 :')
    except ValueError:
        print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
        age = 18
    newMember = Member(name, age, email)
    member_list.append(newMember)
    cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', newMember.as_dic())
    
conn.commit()
print('입력한 자료는 이상 {}명 입니다'.format(len(member_list)))
print('이름\t나이\t메일')
for member in member_list:
    print(member)

이름을 입력해주세요 : 이르음
나이를 입력해주세요 :33
메일을 입력해주세요 :k@l.com
이름을 입력해주세요 : 핸드폰
나이를 입력해주세요 :22
메일을 입력해주세요 :h@h.com
이름을 입력해주세요 : 0
입력한 자료는 이상 2명 입니다
이름	나이	메일
이르음	33	k@l.com
핸드폰	22	h@h.com


In [8]:
# 입력확인은 DB browser 나 python으로 
cursor.execute('SELECT * FROM MEMBER')
cursor.fetchall()

[('홍길동', 20, 'h@h.com'),
 ('신길동', 25, 'S@S.COM'),
 ('신림동', 30, 's@h.com'),
 ('홍낄뚱', 18, 'h@h.com'),
 ('김세동', 33, 'k@k.com'),
 ('김민준', 31, 'fn45@naver.com'),
 ('홍', 2, 'h@h.com'),
 ('이르음', 33, 'k@l.com'),
 ('핸드폰', 22, 'h@h.com')]

# 2절. 오라클 데이터 베이스 연결
- [cx_Oracle](https://cx-oracle.readthedocs.io/en/latest/)

In [9]:
import cx_Oracle

In [11]:
# conn 얻어오는 방법 1
oracle_dsn = cx_Oracle.makedsn(host="localhost", port=1521, sid='xe')
print(oracle_dsn)
conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
conn
# 여기서 예외가 날 경우 VC_redist.x64.exe

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))


<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [12]:
conn.close()

In [13]:
# conn 얻어오는 방법2
conn = cx_Oracle.connect("scott", "tiger", "localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [14]:
# cursor 객체 생성
cursor = conn.cursor()
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()

In [15]:
for e in emp: # 튜플 리스트라 field가 많은 경우 가독성 다소 떨어짐
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [17]:
import pandas as pd
emp_df = pd.DataFrame(emp)
emp_df.head()

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [18]:
# select 문 수행한 필드 정보
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [22]:
result = []
for description in cursor.description:
    result.append(description[0].lower())
result

['empno', 'ename', 'job', 'mgr', 'hiredate', 'sal', 'comm', 'deptno']

In [20]:
[row[0] for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [21]:
emp_df.columns

RangeIndex(start=0, stop=8, step=1)

In [23]:
emp_df.columns = [row[0] for row in cursor.description]
emp_df.sample() # 임의의 한줄 출력

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
12,7902,FORD,ANALYST,7566.0,1981-12-03,3000.0,NaN,20


In [28]:
# 검색할 이름을 사용자에게 받아 해당 내용을 출력
ename = input('검색 하실 이름은 : ')
sql = "SELECT * FROM EMP WHERE ENAME=UPPER(:name)"
cursor.execute(sql, {'name':ename})
emp = cursor.fetchall()
if len(emp):
    for e in emp:
        print(e)
else:
    print("입력하신 이름의 데이터는 없습니다")

검색 하실 이름은 : 곽지성
입력하신 이름의 데이터는 없습니다


In [31]:
fieldnames = [description[0] for description in cursor.description]
print(fieldnames)
print(e)

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)


In [34]:
for idx in range(len(e)):
    print("{}:{}".format(fieldnames[idx], e[idx] if e[idx] is not None else ''))

EMPNO:7788
ENAME:SCOTT
JOB:ANALYST
MGR:7566
HIREDATE:1982-12-09 00:00:00
SAL:3000.0
COMM:
DEPTNO:20


In [36]:
for fieldname, data in zip(fieldnames, e):
    print("{}:{}".format(fieldname, data if data is not None else ''))

EMPNO:7788
ENAME:SCOTT
JOB:ANALYST
MGR:7566
HIREDATE:1982-12-09 00:00:00
SAL:3000.0
COMM:
DEPTNO:20


In [37]:
cursor.close()
conn.close()

# 3절. 연습문제

In [1]:
import sqlite3
conn = sqlite3.connect('data/ch10_data.db')
cursor = conn.cursor()

In [10]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    def as_dict(self): #insert할 때 사용
        return {"name":self.name, "phone":self.phone, 
                "email":self.email, "age":self.age, "grade":self.grade, 
                "etc":self.etc}
    def __str__(self):
        return "{}\t{}\t{}\t{}\t{}\t{}".format(self.name,
                                              self.phone,
                                              self.email,
                                              self.age,
                                              self.grade,
                                              self.etc)
def to_member(*row): #튜플 데이러를 매개변수로 받아 Member형 객체로 return 
    return Member(row[0], row[1], row[2], row[3], row[4], row[5])

In [95]:
dbreadmember = ('홍길동', '010-4570-8382', 'h@h.com', 20, 5, '...')
m = to_member(*dbreadmember) # 튜플 언패킹
print(m)
print(m.as_dic())

홍길동	010-4570-8382	h@h.com	20	5	...
{'name': '홍길동', 'phone': '010-4570-8382', 'email': 'h@h.com', 'age': 20, 'grade': 5, 'etc': '...'}


### 1.입력

In [3]:
def insert_member():
    '사용자로부터 이름, 전화, 이메일, 나이, 등급(1~5), 기타특징 입력받아 DB에 추가한다'
    cursor = conn.cursor()
    name = input('이름 : ')
    phone = input("전화번호 : ")
    email = input("이메일 : ")
    try:
        age = int(input("나이 : "))
        if age<0:
            age = 0
    except ValueError as e:
        print('유효하지 않은 나이  입력시 나이는 0으로 초기화')
        age = 0
    try:
        grade = int(input("고객등급(1~5) : "))
        if grade < 1 :
            grade = 1
        if grade > 5 :
            grade = 5
    except ValueError as e:
        print('유효하지 않은 등급을 입력시 등급은 1로 초기화')
        grade = 1
    etc = input("기타 정보 : ")
    member = Member(name, phone, email, age, grade, etc)
    cursor.execute('''
        INSERT INTO MEMBER VALUES (:name, :phone, :email, :age, :grade, :etc)
    ''', member.as_dict())
    conn.commit()
    print(member, "님 데이터 저장완료")
    cursor.close()   

In [11]:
import sqlite3
conn = sqlite3.connect('data/ch10_data.db')
insert_member()

이름 입력 : 윤길동
핸드폰 번호 입력 : 010-1234-5678
이메일 입력 : y@y.com
나이 입력 : 18
등급 입력 : 5
기타 사항 입력 : 18


### 2. 전체 출력

In [4]:
def select_all():
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MEMBER")
    members = cursor.fetchall()
    print('='*70)
    print("{:^70}".format("고객 정보"))
    print('-'*70)
    print("{:^5}\t{:^3}\t{:^10}{:^15}\t{:^3}\t{}".format("GRADE",
                    "이름", "전화", "메일", "나이", "기타"))
    print('='*70)
    for member in members:
        print(to_member(*member))
    if len(members)==0:
        print("저장된 회원이 없습니다") 
    print('='*70)
    cursor.close()

In [10]:
select_all()

                                고객 정보                                 
----------------------------------------------------------------------
GRADE	이름 	    전화          메일       	나이 	기타
김민준	010-4570-8382	f@f.com	18	5	18
곽지성	010-1234-5678	k@j.com	18	5	18
김지환	010-4571-1234	q@q.com	18	5	18
홍길동	010-1895-1823	i@i.com	18	5	18
홍길동	010-4570-8382	f@f.com	18	5	18
에베베	010-4562-5852	q@p.com	18	5	18


### 3. 이름 검색

In [5]:
def select_name():
    cursor = conn.cursor()
    name = input("조회할 이름 입력 : ")
    sql = "SELECT * FROM MEMBER WHERE NAME=UPPER(:name)"    
    cursor.execute(sql, {'name':name})
    member_list = cursor.fetchall()        
    if len(member_list): 
        for row in member_list:
            member = to_member(*row)
            print(member)  
    else:
        print("조회된 결과가 없습니다.")
    cursor.close()

In [14]:
select_name()

조회할 이름 입력 : 김민준
김민준	010-4570-8382	f@f.com	18	5	18


### 4. 메일 삭제

In [18]:
conn.close()

In [6]:
def delete_email():
    cursor = conn.cursor()
    email = input('삭제하실 이메일 : ')
    cursor.execute('DELETE FROM MEMBER WHERE EMAIL=:email',{'email':email})
    conn.commit()
    if cursor.rowcount > 0:
        print("{} 해당 이메일을 가진 고객을 삭제 했습니다.".format(email))
    else:
        print("{} 해당 이메일을 가진 고객을 찾을수 없습니다".format(email))
    cursor.close()

In [ ]:
delete_email()

### 5. csv 내보내기

In [7]:
def save_csv():
    'sql 실행 결과를 튜플 리스트로 받아와서 db결과 title(description)과 튜플을 csv에 출력'
    import csv
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM MEMBER')
    members = cursor.fetchall()
    fieldnames = [description[0] for description in cursor.description ]
    cursor.close()
    file_name = input('파일명을 입력하세요(csv 확장자 포함)')
    with open('data/'+file_name, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)
        writer.writerows(members)

### 6. 종료

In [8]:
def close_sql():
    global conn
    conn.close()
    print("종료합니다")

### main

In [11]:
def main():
    while True:
        print("1.입력", "2.전체 조회", "3.이름 찾기", "4.메일 삭제", "5.CSV 내보내기", "0.종료",
             sep=" | ")
        try:
            menu = int(input("메뉴 선택: "))
        except:
            print("유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.")
        if menu==1:
            insert_member()
        elif menu==2:
            select_all()
        elif menu==3:
            select_name()
        elif menu==4:
            delete_email()
        elif menu==5:
            save_csv()
        elif menu==0:
            close_sql()
            break

if __name__=='__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data/ch10_data.db')
    main()            

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 1
이름 : 다다다
전화번호 : 18
이메일 : 18
나이 : 18
고객등급(1~5) : 5
기타 정보 : 18
다다다	18	18	18	5	18 님 데이터 저장완료
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 2
                                고객 정보                                 
----------------------------------------------------------------------
GRADE	이름 	    전화          메일       	나이 	기타
곽지성	010-1234-5678	k@j.com	18	5	18
김지환	010-4571-1234	q@q.com	18	5	18
홍길동	010-1895-1823	i@i.com	18	5	18
에베베	010-4562-5852	q@p.com	18	5	18
데데데	010-4156-4562	18	18	18	18
다다다	18	18	18	5	18
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 3
조회할 이름 입력 : 다다다
다다다	18	18	18	5	18
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 4
삭제하실 이메일 : 18
18 해당 이메일을 가진 고객을 삭제 했습니다.
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 5
파일명을 입력하세요(csv 확장자 포함)test
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 0
종료합니다


In [ ]:
cursor.execute('''
    CREATE TABLE MEMBER(
    NAME VARCHAR2(30),
    PHONE VARCHAR2(30),
    EMAIL VARCHAR2(30),
    AGE NUMBER(2),
    GRADE NUMBER(2),
    ETC VARCHAR2(30)
)
''')